# **Estimación del valor de una casa con una red neuronal densa**

Lo primero que haremos será importar las librerías que vamos a utilizar,
cargar el dataset y mostrar las estadísticas de frecuencias de clases:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [ ]:
# Cargamos California Housing Dataset

data = fetch_california_housing()

In [ ]:
X = data.data # variables predictoras (son 8)
y = data.target.reshape(-1,1) # variable target

In [ ]:
# Para obtener detalles sobre la construcción del dataset
# y sus variables:

print(data["DESCR"])

In [ ]:
# Separamos los datos en training (70%) y test (30%)

X_tr, X_te, y_tr, y_te = train_test_split(X, y,
                                          test_size=0.3,
                                          random_state=1)

In [ ]:
# Separamos a su vez training en training final (70%) y
# validación (30%)

X_tr, X_va, y_tr, y_va = train_test_split(X_tr, y_tr,
                                          test_size=0.3,
                                          random_state=2)

In [ ]:
print(X_tr.shape)
print(X_va.shape)
print(X_te.shape)

Como podemos observar, hay ocho variables predictoras.
Por las dimensiones
de los arrays vemos que en training hay 10113 viviendas, en validación 4335 y en test 6192.
Ahora chequearemos la media y desviación estándar en training de las ocho variables predictoras:

In [ ]:
print(X_tr.mean(axis=0).round(2))

In [ ]:
print(X_tr.std(axis=0).round(2))

Vemos que ni los promedios son cercanos a 0 ni las desviaciones estándar próximas a 1, ni parecidas (de hecho, hay diferencias de 3 órdenes de magnitud entre algún par de ellas).
Por tanto, necesitamos estandarizar las variables predictoras:

In [ ]:
from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()
sc_x.fit(X_tr)
X_tr_sc = sc_x.transform(X_tr)
X_va_sc = sc_x.transform(X_va)
X_te_sc = sc_x.transform(X_te)

Si ahora calculamos las medias y desviaciones en entrenamiento, obtenemos:

In [ ]:
print(X_tr_sc.mean(axis=0).round(2))

In [ ]:
print(X_tr_sc.std(axis=0).round(2))

Por lo que ahora todas las medias en entrenamiento son 0, y todas las desviaciones 1.

Al ser un problema de regresión, tenemos que comprobar si la variable target (el valor de la casa) hay que normalizarlo también.
Si calculamos la media y desviación estándar de la variable target en entrenamiento:

In [ ]:
print(y_tr.mean(axis=0).round(2))

In [ ]:
print(y_tr.std(axis=0).round(2))

Ahora estandarizamos la variable target y calculamos la media y desviación estándar de la versión estandarizada:

In [ ]:
sc_y = StandardScaler()
sc_y.fit(y_tr)
y_tr_sc = sc_y.transform(y_tr)
y_va_sc = sc_y.transform(y_va)
y_te_sc = sc_y.transform(y_te)

In [ ]:
print(y_tr_sc.mean(axis=0).round(2))

In [ ]:
print(y_tr_sc.std(axis=0).round(2))

Vemos que la media de y_tr_sc en entrenamiento es 0, y su desviación estándar 1.

El siguiente paso es definir la red neuronal.
En la capa de entrada (Input) se especifica el número de variables predictoras, ocho.
A continuación, se define una capa oculta de 5 neuronas con función de activación no lineal tipo ReLU, y una capa de salida de una neurona, ya que deseamos que la red prediga un único valor, que es el precio estimado de la casa.

In [ ]:
model = keras.Sequential([
    layers.Input(shape=8),
    layers.Dense(units=5, activation="relu",
                 name="Densa_oculta"),
    layers.Dense(units=1, activation="linear",
                 name="Salida")
])

A continuación, realizamos la operación de compilación de la red, en la cual especificamos el tipo de optimizador (RMSPprop con un learning rate por defecto de 1e-3) y la función de pérdida (Error Cuadrático Medio, “MSE”).

In [ ]:
model.compile(optimizer="rmsprop", loss="mse")

La operación summary permite extraer el resumen de las capas formadas por la red, la dimensión de su información de salida, y el número de parámetros:

In [ ]:
model.summary()

Otra forma de representar el flujo de información en la red que hemos definido es la siguiente:

In [ ]:
plot_model(model, "esquema.png", show_shapes=True,
           #dpi=200 # para una mayor resolución
          )

Lo siguiente que especificaremos serán los callbacks, que son operaciones que serán lanzadas en el transcurso del entrenamiento de la red.
El primero de ellos es un ModelCheckpoint, que al final de cada época graba a fichero los pesos de la red si la métrica que le indiquemos ha obtenido el mejor valor hasta el momento.
En nuestro caso, monitorizaremos el loss en validación.
Por tanto, al final del proceso de entrenamiento tendremos en fichero los pesos de la red con mejor métrica en validación durante este proceso.
El segundo callback que definiremos será un EarlyStopping, que detiene el entrenamiento de la red si hay varias épocas seguidas sin que mejore la métrica que le indiquemos.
En nuestro caso, queremos que el entrenamiento se detenga si durante 5 épocas no se produce una mejora en el accuracy de validación.

In [ ]:
lista_callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="best_model.keras",
        monitor="val_loss",
        save_best_only=True,
    )
]

A continuación, entrenamos la red durante un máximo de 100 épocas, ya que el entrenamiento se podría detener antes por el callback de early stopping.
La salida de la llamada a model.fit la recogemos en la variable “historia”, que contendrá la evolución en el proceso de entrenamiento de las diferentes métricas.

In [ ]:
historia = model.fit(X_tr_sc, y_tr_sc,
                     epochs=100, batch_size=64,
                     callbacks=lista_callbacks,
                     validation_data=(X_va_sc, y_va_sc))

Ahora representamos gráficamente la evolución del entrenamiento de la red:

In [ ]:
from matplotlib.ticker import MaxNLocator

f = plt.figure(figsize=(4,4))
h = historia.history
mejor_epoca = np.argmin(h["val_loss"])
plt.plot(h["loss"], label="entrenamiento")
plt.plot(h["val_loss"], label="validación")
plt.plot(mejor_epoca, h["val_loss"][mejor_epoca], "or")
plt.title("Loss", fontsize=18)
plt.xlabel("Época", fontsize=18)
plt.xticks(fontsize=12); plt.yticks(fontsize=12)
plt.legend()
f.gca().xaxis.set_major_locator(MaxNLocator(integer=True));

Como podemos ver en las gráficas, se han entrenado menos épocas de las 100 inicialmente programadas, ya que el mecanismo de parada temprana (early stopping) ha detenido el proceso al haber detectado que la tasa de acierto en validación no mejoraba durante 5 épocas seguidas.
Por otra parte, el punto rojo está marcando la época en la que se ha obtenido un mejor loss en validación.
Recordemos que en el fichero tenemos los pesos de la red en ese momento.
Los recuperamos:

In [ ]:
model = keras.models.load_model("best_model.keras")

Y ahora lanzaremos la predicción en las cinco primeras viviendas de test:

In [ ]:
print(model.predict(X_te_sc[:5]))

Es muy importante notar que, ya que hemos entrenado la red con los datos estandarizados, debemos lanzar nuestras nuevas predicciones con datos estandarizados también (X_te_sc).
Recordemos que esos datos se normalizaron usando las estadísticas de training.
Lo que observamos es que los precios estimados pueden ser negativos.
Esto sucede porque hemos entrenado también con los datos del target normalizado, por lo que nuestro modelo predice precios de casas
normalizados.
Cero indica una predicción igual a la media en entrenamiento, por lo que un valor positivo indica un precio mayor que la media, y un valor negativo indica un precio menor que la media.

Por esto, para obtener el precio predicho por nuestro modelo en la escala original del dataset debemos deshacer la estandarización:

In [ ]:
print(sc_y.inverse_transform(model.predict(X_te_sc[:5])))

Observamos que ahora las estimaciones están en la escala original.

Finalmente, evaluamos la calidad de nuestro modelo de regresión mediante el coeficiente de determinación R2:

In [ ]:
from sklearn.metrics import r2_score as R2_score

In [ ]:
y_tr_p = sc_y.inverse_transform(model.predict(X_tr_sc))
R2_tr = R2_score(y_tr, y_tr_p)
print("R2 en training: {:.2f}".format(R2_tr)) 

In [ ]:
y_va_p = sc_y.inverse_transform(model.predict(X_va_sc))
R2_va = R2_score(y_va, y_va_p)
print("R2 en validación: {:.2f}".format(R2_va)) 

In [ ]:
y_te_p = sc_y.inverse_transform(model.predict(X_te_sc))
R2_te = R2_score(y_te, y_te_p)
print("R2 en test: {:.2f}".format(R2_te))